This jupyter notebook is to show function `babs-init`.

In [1]:
# See `README_howtorun_show_babs_init_ipynb.md` for how to run this file!

# Clone out:
#   datalad clone ria+file:///path/to/<container>/output_ria#~data singleORmulti-ses_<container>_outputs
# If you want to change `project_root`'s foldername, also change `output_ria/alias/data` symlink:
    # cd output_ria/alias
    # ln -sf /new/full/path/to/output_ria/xxx/xxx-xxx-xxx-xxx data

# flake8: noqa

# set up how Jupyter notebook behaves:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'   # print all outputs

import os
import os.path as op
import warnings

There are several arguments when calling `babs-init`:

In [3]:
# This notebook only accepts toybidsapp_rawBIDS, qsiprep or fmriprep
#   or fmriprep_anatonly or 'fmriprep_sloppyFlag' or fmriprepfake (input ds: BIDS)
# ++++++++++++++++++++++++++++++++++
bidsapp = "xcpd"
bidsapp_version_dash = "0-4-0"
# xcp-d: '0-3-0'; '0-4-0'

# fmriprep_version_dash = "22-0-2"
# fmriprep output: '22-0-2' for HBN fmriprep func outputs; '20.2.3' for CCNP fmriprep outputs

task_name = "nifti"    # "nifti"

input_ds_name = "fmriprep"    # "fmriprep"

type_session = "single-ses"
which_dataset = "CCNP"    # "HBN"
input_ds = None    # None
# ^^ `None`` if `which_dataset` not 'customize'; please provide the path if `which_dataset` is 'customize'

flag_where = "cubic"     # "cubic" or "local" or "msi"
type_system = "sge"   # "sge" or "slurm"
# ++++++++++++++++++++++++++++++++++

# sanity checks:
if flag_where == "cubic":
    assert type_system == "sge"
elif flag_where == "msi":
    assert type_system == "slurm"

# where:
if flag_where == "cubic":
    where_root = "/cbica/projects/BABS"
    where_project = op.join(where_root, "data")
    where_notebooks = op.join(where_root, "babs/notebooks")
elif flag_where == "local":
    where_root = "/Users/chenyzh/Desktop/Research/Satterthwaite_Lab/datalad_wrapper"
    where_project = op.join(where_root, "data")
    where_notebooks = op.join(where_root, "babs/notebooks")
elif flag_where == "msi":
    where_root = "/home/faird/zhaoc"
    where_project = "/home/faird/zhaoc/data"
    where_notebooks = op.join(where_root, "babs/notebooks")
else:
    raise Exception("not valid `flag_where`!")

# Input dataset:
if which_dataset == "HBN":  # HBN data:
    assert type_session == "single-ses"
    if flag_where == "cubic":
        input_ds = "/cbica/projects/BABS/data/HBN_fmriprep_func"  # datalad sibling in BABS cubic project
elif which_dataset == "CCNP":   # CCNP data:
    assert type_session == "single-ses"
    if flag_where == "cubic":
        input_ds = "/cbica/projects/BABS/data/CCNP_fmriprep"
project_name = "test_babs_" + type_session
# Based on which dataset:
if which_dataset in ["HBN", "PNC", "CCNP", "customize"]:   # specific real dataset
    project_name += "_" + which_dataset
# Based on which BIDS App:
assert bidsapp == "xcpd"
container_name = bidsapp + "-" + bidsapp_version_dash

project_name += "_" + bidsapp + "_" + task_name

# Container:
container_ds = op.join(where_project, bidsapp + "-container")  
if flag_where == "local":
    # container_ds += "-docker"   # add "docker" at the end
    container_ds = op.join(where_project, "toybidsapp-container-docker")

container_config_yaml_file = op.join(where_notebooks, "eg_" + container_name
    + "_" + task_name + "_" + type_system)
if flag_where in ["cubic", "msi"]:
    container_config_yaml_file += "_" + flag_where
else:
    if type_system == "sge":
        container_config_yaml_file += "_" + "cubic"
    elif type_system == "slurm":
        container_config_yaml_file += "_" + "msi"
container_config_yaml_file += ".yaml"
print(container_config_yaml_file)
assert op.exists(container_config_yaml_file)
# container_config_yaml_file = op.join(where_notebooks, "example_container_" + flag_instance + ".yaml")

# list_sub_file = op.join(where_notebooks, "initial_sub_list_" + type_session + ".csv")
list_sub_file = None

/cbica/projects/BABS/babs/notebooks/eg_xcpd-0-4-0_nifti_sge_cubic.yaml


Let's pass these arguments into `babs-init` CLI:

In [4]:
cmd = "babs-init \\\n"
cmd += "\t" + "--where_project " + where_project + " \\\n"
cmd += "\t" + "--project_name " + project_name + " \\\n"
cmd += "\t" + "--input " + input_ds_name + " " + input_ds + " \\\n"
if list_sub_file is not None:
    cmd += "\t" + "--list_sub_file " + list_sub_file + " \\\n"
cmd += "\t" + "--container_ds " + container_ds + " \\\n"
cmd += "\t" + "--container_name " + container_name + " \\\n"
cmd += "\t" + "--container_config_yaml_file " + container_config_yaml_file + " \\\n"
cmd += "\t" + "--type_session " + type_session + " \\\n"
cmd += "\t" + "--type_system " + type_system

print("The command to execute:")
print(cmd)
print("WARNING: make sure you've changed `--fs-license-file` value in YAML file if you use it!!!")

The command to execute:
babs-init \
	--where_project /cbica/projects/BABS/data \
	--project_name test_babs_single-ses_CCNP_xcpd_nifti \
	--input fmriprep /cbica/projects/BABS/data/CCNP_fmriprep \
	--container_ds /cbica/projects/BABS/data/xcpd-container \
	--container_name xcpd-0-4-0 \
	--container_config_yaml_file /cbica/projects/BABS/babs/notebooks/eg_xcpd-0-4-0_nifti_sge_cubic.yaml \
	--type_session single-ses \
	--type_system sge


In [ ]:
!($cmd)